In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import warnings

Load in the data from csv (put dataset in same directory as this file)

In [2]:
data = pd.read_csv("train.csv")
print(data.shape) # (891, 12)

(891, 12)


# Helper Functions

### Function to split data into k folds

In [3]:
# function for splitting dataset into k folds
def split_data(y, folds):
    # split data into k equal (as possible) sized sets
    np.random.seed(0)
    L = np.size(y)
    order = np.random.permutation(L)
    c = np.ones((L,1))
    L = int(np.round((1/folds)*L))
    for s in np.arange(0,folds-2):
        c[order[s*L:(s+1)*L]] = s + 2
    c[order[(folds-1)*L:]] = folds
    return c

### PCA Normalization

In [4]:
# compute eigendecomposition of data covariance matrix for PCA transformation
def PCA(x,**kwargs):
	# regularization parameter for numerical stability
	lam = 10**(-7)
	if 'lam' in kwargs:
		lam = kwargs['lam']

	# create the correlation matrix
	P = float(x.shape[1])
	Cov = 1/P*np.dot(x,x.T) + lam*np.eye(x.shape[0])

	# use numpy function to compute eigenvalues / vectors of correlation matrix
	d,V = np.linalg.eigh(Cov)
	return d,V

# PCA-sphereing - use PCA to normalize input features
def PCA_sphereing(x,**kwargs):
	# Step 1: mean-center the data
	x_means = np.mean(x, axis=1, dtype=float)[:,np.newaxis]
	x_centered = x - x_means

	# Step 2: compute pca transform on mean-centered data
	d,V = PCA(x_centered,**kwargs)

	# Step 3: divide off standard deviation of each (transformed) input,
	# which are equal to the returned eigenvalues in 'd'.
	stds = (d[:,np.newaxis])**(0.5)

	# check to make sure thta x_stds > small threshold, for those not
	# divide by 1 instead of original standard deviation
	ind = np.argwhere(stds < 10**(-2))
	if len(ind) > 0:
		ind = [v[0] for v in ind]
		adjust = np.zeros((stds.shape))
		adjust[ind] = 1.0
		stds += adjust

	normalizer = lambda data: np.dot(V.T,data - x_means)/stds

	# create inverse normalizer
	inverse_normalizer = lambda data: np.dot(V,data*stds) + x_means

	# return normalizer
	return normalizer,inverse_normalizer

### Standard Normalizer

In [5]:
def standard_normalizer(x):    
    # compute the mean and standard deviation of the input
    x_means = np.nanmean(x,axis=0)[:,np.newaxis]
    x_stds = np.nanstd(x,axis=0, dtype=float)[:,np.newaxis]   

    # check to make sure thta x_stds > small threshold, for those not
    # divide by 1 instead of original standard deviation
    ind = np.argwhere(x_stds < 10**(-2))
    if len(ind) > 0:
        ind = [v[0] for v in ind]
        adjust = np.zeros((x_stds.shape))
        adjust[ind] = 1.0
        x_stds += adjust

    # fill in any nan values with means 
    ind = np.argwhere(np.isnan(x) == True)
    for i in ind:
        x[i[0],i[1]] = x_means[i[0]]

    # create standard normalizer function
    normalizer = lambda data: (data - x_means)/x_stds

    # create inverse standard normalizer
    inverse_normalizer = lambda data: data*x_stds + x_means

    # return normalizer 
    return normalizer,inverse_normalizer

### Accuracy

In [6]:
def accuracy(pred, actual):
    df = pd.DataFrame({'Predicted': pred, 'Actual': actual})
    df = df.reset_index()

    correct = 0
    for index, row in df.iterrows():
        if row['Predicted'] == row['Actual']:
            correct += 1

    return correct / float(len(df.index))

# Testing different methods of pre-processing
Original data, standard normalization, and PCA normalization

In [7]:
loops = 100

for normalizer in range(3):
    total_acc = [0,0,0]
    if normalizer == 0:
        print("Original Data")
    if normalizer == 1:
        print("Standard Normalized")
    if normalizer == 2:
        print("PCA Normalized")

    for i in range(loops):
        # These are the features that we want to use for training,
        # since some of them would not be relevant or contain incomplete data
        features = ["Pclass", "Sex", "SibSp", "Parch"]

        # Every loop, we change the random_state so that splits are random and reproducible
        train_split, test_split = train_test_split(data, test_size=0.2, random_state=i)
        
        # get_dummies assigns numerical values to non-numerical data
        x_train = pd.get_dummies(train_split[features])
        x_test = pd.get_dummies(test_split[features])

        # Standard Normalization
        if normalizer == 1:
            std, rev_std = standard_normalizer(x_train)
            x_train = std(np.array(x_train).T).T
            x_test = std(np.array(x_test).T).T

        # PCA Normalization
        if normalizer == 2:
            std, rev_std = PCA_sphereing(np.array(x_train, dtype=float).T)
            x_train = std(np.array(x_train).T).T
            x_test = std(np.array(x_test).T).T

        y_train = np.array(train_split['Survived'])
        y_test = np.array(test_split['Survived'])

        ### Model 1: Kernel-based method
        model1 = svm.SVC(kernel='rbf')
        model1.fit(x_train, y_train)
        pred1 = model1.predict(x_test)

        model1_accuracy = accuracy(pred1, y_test)
        total_acc[0] += model1_accuracy

        ### Model 2: Neural Network
        model2 = MLPClassifier(hidden_layer_sizes=(25,), alpha=0.001, max_iter=1000, random_state=0)
        model2.fit(x_train, y_train)
        pred2 = model2.predict(x_test)

        model2_accuracy = accuracy(pred2, y_test)
        total_acc[1] += model2_accuracy

        # Model 3: Decision Tree
        model3 = DecisionTreeClassifier()
        model3.fit(x_train, y_train)
        pred3 = model2.predict(x_test)

        model3_accuracy = accuracy(pred3, y_test)
        total_acc[2] += model3_accuracy

    print("Kernel:", total_acc[0] / loops)
    print("NN:    ", total_acc[1] / loops)
    print("Tree:  ", total_acc[2] / loops)
    print()

Original Data
Kernel: 0.8032402234636871
NN:     0.7986033519553075
Tree:   0.7986033519553075

Standard Normalized
Kernel: 0.8032960893854749
NN:     0.7995530726256987
Tree:   0.7995530726256987

PCA Normalized
Kernel: 0.7985474860335197
NN:     0.8012290502793298
Tree:   0.8012290502793298



The accuracies are very similar, so we will pick original data as it requires the least amount of processing.

# K-Fold Validation on NN

In [8]:
# Loop through different hyperparameters, saving the ones with lowest validation cost
best_accuracy = 0
best_hyperparams = []

num_folds = 4
folds = split_data(y_train, num_folds)

for exp1 in range(3, 8):
    alpha = 10**(-exp1)
    for exp2 in range(3, 8):
        lam = 10**(-exp2)
        lam = 10**(-exp2)
        total_val_accuracy = 0
        # print(f"Testing alpha={alpha}, lambda={lam}")

        for i in range(num_folds):
            # run gradient descent with current alpha and lam
            # on every fold except i
            current_train_x = []
            current_train_y = []

            current_val_x = []
            current_val_y = []

            for j, f in enumerate(folds):
                if int(f) - 1 == i:
                    current_val_x.append(x_train[j])
                    current_val_y.append(y_train[j])
                else:
                    current_train_x.append(x_train[j])
                    current_train_y.append(y_train[j])
            
            current_train_x = np.array(current_train_x)
            current_train_y = np.array(current_train_y)

            current_val_x = np.array(current_val_x)
            current_val_y = np.array(current_val_y)

            warnings.simplefilter('ignore', category=ConvergenceWarning)
            model2 = MLPClassifier(hidden_layer_sizes=(25,), learning_rate_init=alpha, alpha=lam, max_iter=1000, random_state=0)
            model2.fit(current_train_x, current_train_y)
            pred2 = model2.predict(current_val_x)

            # calculate accuracy of fold i using predicted values
            total_val_accuracy += accuracy(pred2, current_val_y)
        
        # if better than current best, save
        avg_val_accuracy = total_val_accuracy / float(num_folds)

        if avg_val_accuracy > best_accuracy:
            best_accuracy = avg_val_accuracy
            best_hyperparams = [alpha, lam]

print(f"Best Validation Accuracy: {best_accuracy}")
print(f"Best Hyperparameters: {best_hyperparams}\n")

Best Validation Accuracy: 0.8160112359550561
Best Hyperparameters: [0.001, 0.001]



Optimal hyperparameters:
- Learning Rate: 0.001
- Regularization Parameter: 0.001

Test the best hyperparameters determined by k-fold validation

In [9]:
learning_rate = 0.001
alpha = 0.001
warnings.simplefilter('ignore', category=ConvergenceWarning)
model2 = MLPClassifier(hidden_layer_sizes=(25,), learning_rate_init=learning_rate, alpha=alpha, max_iter=1000, random_state=0)
model2.fit(x_train, y_train)
pred2 = model2.predict(x_test)
print(accuracy(pred2, y_test))

0.770949720670391


# Test if adding or removing features improves the result

In [10]:
features_list = [
    "Pclass",
    "Sex",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked"
]

loops = 100
top_features = []

for _ in range(6):

    best_feature = ""
    best_accuracy = 0

    for feature in features_list:
        total_acc = [0,0,0]

        if feature in top_features:
            continue

        features = top_features + [feature]

        for i in range(loops):
            # Every loop, we change the random_state so that splits are random and reproducible
            train_split, test_split = train_test_split(data, test_size=0.2, random_state=i)
            # print(train_split.shape)
            
            # get_dummies assigns numerical values to non-numerical data
            x_train = pd.get_dummies(train_split[features])
            x_test = pd.get_dummies(test_split[features])
            # print(x_train.shape)

            # Standard Normalization
            if normalizer == 1:
                std, rev_std = standard_normalizer(x_train)
                x_train = std(np.array(x_train).T).T
                x_test = std(np.array(x_test).T).T

            # PCA Normalization
            if normalizer == 2:
                std, rev_std = PCA_sphereing(np.array(x_train, dtype=float).T)
                x_train = std(np.array(x_train).T).T
                x_test = std(np.array(x_test).T).T

            y_train = np.array(train_split['Survived'])
            y_test = np.array(test_split['Survived'])

            ### Model 1: Kernel-based method
            model1 = svm.SVC(kernel='rbf')
            model1.fit(x_train, y_train)
            pred1 = model1.predict(x_test)

            model1_accuracy = accuracy(pred1, y_test)
            total_acc[0] += model1_accuracy

            ### Model 2: Neural Network
            model2 = MLPClassifier(hidden_layer_sizes=(25,), alpha=0.001, max_iter=1000, random_state=0)
            model2.fit(x_train, y_train)
            pred2 = model2.predict(x_test)

            model2_accuracy = accuracy(pred2, y_test)
            total_acc[1] += model2_accuracy

            # Model 3: Decision Tree
            model3 = DecisionTreeClassifier()
            model3.fit(x_train, y_train)
            pred3 = model2.predict(x_test)

            model3_accuracy = accuracy(pred3, y_test)
            total_acc[2] += model3_accuracy

        avg_acc = sum(total_acc) / (3 * loops)
        if avg_acc > best_accuracy:
            best_accuracy = avg_acc
            best_feature = feature

    top_features.append(best_feature)
    print(top_features)
    print(best_accuracy)

['Sex']
0.7875977653631288
['Sex', 'SibSp']
0.7963873370577284
['Sex', 'SibSp', 'Parch']
0.7972439478584732
['Sex', 'SibSp', 'Parch', 'Embarked']
0.8013966480446931
['Sex', 'SibSp', 'Parch', 'Embarked', 'Pclass']
0.8025512104283056
['Sex', 'SibSp', 'Parch', 'Embarked', 'Pclass', 'Fare']
0.8004469273743017
